In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage.measure as sime

In [ ]:
im = cv2.imread('/content/drive/MyDrive/obrazy/projekt/elementy1_1.tif')

In [ ]:
# split image to channels

imageChannels = cv2.split(im)

# thresholding channels
for i in range(0, 3):
    _, imageChannels[i] = cv2.threshold(imageChannels[i], 60, 255, cv2.THRESH_BINARY)
    imageChannels[i] = np.reshape(
        imageChannels[i], newshape=(imageChannels[0].shape[0], imageChannels[0].shape[1], 1))

# concatenate channels
image = np.concatenate((imageChannels[0], imageChannels[1], imageChannels[2]), axis=2)

# OTSU threshold - for binary image as a reference
_, binaryImage = cv2.threshold(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_OTSU)


In [ ]:
binaryImage = ~binaryImage

# labelling
labels = sime.label(binaryImage)

# feature extraction
features = sime.regionprops(labels)

# counting colors
colors = np.unique(image.reshape(-1, image.shape[2]), axis=0) # colors in the image

colors[1,:] = colors[2,:]
colors[2,:] = colors[3,:]

bboxCoordinates = []
objectsNumber = len(features)
categoriesNumber = len(colors)
featuresList = ['EulerNumber', 'Area']
featuresNumber = len(featuresList)
featuresTable = np.zeros((objectsNumber, featuresNumber + 1))
featuresTable[0,:] = np.nan # features nonexistent object with label "0"

for i in range(objectsNumber):
    for j in range(featuresNumber):
      featuresTable[i,j] = features[i][featuresList[j]]
    # color sampling - taking one pixel from each object
    x, y = features[i]['Coordinates'][1] 
    for k in range(categoriesNumber):
        if list(image[x,y,:]) == list(colors[k, :]):
            break;
    featuresTable[i,-1] = k
    bboxCoordinates.append(features[i]['BoundingBox'])

In [ ]:
# check if object is touching edge

def isNotOnEdge(coordinatesList, imageShape):
  notOnEdgeList = []
  for coordinates in coordinatesList:
    if coordinates[0] == 0 or coordinates[1] == 0 or coordinates[2] == imageShape[0] or coordinates[3] == imageShape[1]:
      notOnEdgeList.append(False)
    else:
      notOnEdgeList.append(True) 
  return notOnEdgeList

In [ ]:
# all criteria

notOnEdge = np.where(isNotOnEdge(bboxCoordinates, image.shape))
areaOfSurfaceGroundTruth = 2100

for i in range(3):
  colorName = "green" if i == 0 else "blue" if i == 1 else "pink"
  color = np.where(featuresTable[:,-1] == i)
  imageInColor = np.where(np.isin(labels, color + np.array(1)), 255, 0)

  correctAreaOfSurface = np.where(featuresTable[:,1]  > areaOfSurfaceGroundTruth) 
  euler0 = np.where(featuresTable[:,0] == 0)   

  buffer1 = np.intersect1d(notOnEdge, color)
  buffer2 = np.intersect1d(buffer1, correctAreaOfSurface)
  buffer3 = np.intersect1d(buffer2, euler0)

  info = "Color {} \n"
  print(info.format(colorName))
  print("---------------------------------------------")
  print("Number of items matching the criteria:", buffer3.size)
  print("Number of items not matching the criteria:", buffer1.size - buffer3.size)
  print("---------------------------------------------\n")
